In [2]:
import numpy as np
import pandas as pd
import csv
from scipy import stats
import os
import datetime
import statsmodels.api as sm
import requests
import sys, json
import time
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import random

C:\Users\aclou\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\aclou\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [3]:
# IN CASE NEED TO UPDATE CHROMEDRIVER, RUN BELOW 

# from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager
# driver = webdriver.Chrome(ChromeDriverManager().install())

In [4]:
# DATA PREP FOR DISTILLER SCRAPE 

ls_whisk_name = []
ls_whisk_type = []
ls_whisk_loc = []
ls_comm_rating = []
ls_distiller_score = []
ls_whisk_url = []

# https://distiller.com/search?country_id=1&page=10&sort=total_num_of_ratings&spirit_style_id=1 # THIS IS SINGLE MALT
# spirit style --> 56-63 --> the rums

### Loop through EVERY rum type based on History

We can do this in one ls_url list b/c ls_url can be extended

Note, for rum, we don't do any filtering. Just pick up all rums by first clicking on the rum button when visiting distiller.com home page and then going to the search page

In [14]:
### Start with empty list
ls_url = []

In [15]:
### All Brandies in One Go!

ls_sites = list(range(1, 75)) # FIGURE THIS OUT MANUALLY FOR EACH PULL WE DO
ls_url_iter = [f'https://distiller.com/search?page={str(i)}&sort=total_num_of_ratings' for i in ls_sites]
ls_url.extend(ls_url_iter)

In [16]:
# Print the first page for recon
print(len(ls_url))
ls_url[53]

74


'https://distiller.com/search?page=54&sort=total_num_of_ratings'

In [17]:
### THE BELOW SCRAPER GETS US TO BRANDY

driver = webdriver.Chrome(r'C:\Users\aclou\AppData\Local\Programs\Python\Python39\Scripts\chromedriver')
driver.maximize_window()
driver.get('https://distiller.com')
time.sleep(3)
xpath_brandy = "//div[@class='center-column']/ul[@class='spirit-family-select__options']/li[@class='spirit-family-select__option brandy']/button"
elem = driver.find_element_by_xpath(xpath_brandy)
elem.click() ### this gets us to the BRANDIES

In [18]:
%%time

start = time.time() 

for idx in range(len(ls_url)):
    print(idx)
    print(ls_url[idx])
    url = ls_url[idx]
    driver.get(url)
    time.sleep(1)
    xpath_search_ct = "//div[@class='pagination-control']/span[@class='pagination-control__description']"
    
    # scrape
    xpath_olist = "//ol[@class='spirits']/li[@class='spirit brandy-content']"
    whiskey_elems = driver.find_elements_by_xpath(xpath_olist)
    if len(whiskey_elems) == 0: 
        time.sleep(1)
        pass
    else:
        for i in range(len(whiskey_elems)): 
            elem = whiskey_elems[i]

            whisk_name = elem.find_element_by_xpath(".//div[@class='name']").text
            ls_whisk_name.append(whisk_name)

            whisk_type = elem.find_element_by_xpath(".//span[@class='accented-text']").text
            whisk_type = re.sub(',', '', whisk_type)
            ls_whisk_type.append(whisk_type)

            whisk_loc = elem.find_element_by_xpath(".//span[@class='location']").text
            ls_whisk_loc.append(whisk_loc)
            
            try: 
                comm_rating = elem.find_element_by_xpath(".//div[@class='community-rating']").text
                ls_comm_rating.append(comm_rating)
            except NoSuchElementException:
                comm_rating = float("NaN")
                ls_comm_rating.append(comm_rating)

            try:
                distiller_score = elem.find_element_by_xpath(".//div[@class='distiller-score']").text
            except NoSuchElementException:
                distiller_score = float("NaN")
            ls_distiller_score.append(distiller_score)

            whisk_url = elem.find_element_by_xpath("./a").get_attribute('href')
            ls_whisk_url.append(whisk_url)

            wait_time = random.randrange(10, 20)/10 ### PER REVIEW, TAKE 1s - 2s!!
            time.sleep(wait_time)



0
https://distiller.com/search?page=1&sort=total_num_of_ratings
1
https://distiller.com/search?page=2&sort=total_num_of_ratings
2
https://distiller.com/search?page=3&sort=total_num_of_ratings
3
https://distiller.com/search?page=4&sort=total_num_of_ratings
4
https://distiller.com/search?page=5&sort=total_num_of_ratings
5
https://distiller.com/search?page=6&sort=total_num_of_ratings
6
https://distiller.com/search?page=7&sort=total_num_of_ratings
7
https://distiller.com/search?page=8&sort=total_num_of_ratings
8
https://distiller.com/search?page=9&sort=total_num_of_ratings
9
https://distiller.com/search?page=10&sort=total_num_of_ratings
10
https://distiller.com/search?page=11&sort=total_num_of_ratings
11
https://distiller.com/search?page=12&sort=total_num_of_ratings
12
https://distiller.com/search?page=13&sort=total_num_of_ratings
13
https://distiller.com/search?page=14&sort=total_num_of_ratings
14
https://distiller.com/search?page=15&sort=total_num_of_ratings
15
https://distiller.com/sear

In [19]:
driver.close()

In [20]:
# content = driver.page_source
# whiskey_elems[0].get_attribute('innerHTML')
# print(whiskey_elems[0].text)

In [21]:
list_for_df = [ls_whisk_name, ls_whisk_type, ls_whisk_loc, ls_comm_rating, ls_distiller_score, ls_whisk_url]

df_alc_main_char = pd.DataFrame(np.array(list_for_df).T, \
    columns = ['Alc_Name', 'Alc_Type', 'Alc_Loc', 'Communal_Rating', 'Distiller_Score', 'URL'])
df_alc_main_char['Spirit'] = 'Brandy'
df_alc_main_char.head()

,Alc_Name,Alc_Type,Alc_Loc,Communal_Rating,Distiller_Score,URL,Spirit
0,HENNESSY VS COGNAC,Cognac,"Cognac, France",3.22,78,https://distiller.com/spirits/hennessy-vs-cognac,Brandy
1,RÉMY MARTIN VSOP COGNAC,Cognac,"Cognac, France",3.42,83,https://distiller.com/spirits/remy-martin-vsop...,Brandy
2,RÉMY MARTIN 1738 ACCORD ROYAL COGNAC,Cognac,"Cognac, France",4.12,90,https://distiller.com/spirits/remy-martin-1738...,Brandy
3,HENNESSY PRIVILÈGE VSOP COGNAC,Cognac,"Cognac, France",3.81,85,https://distiller.com/spirits/hennessy-privile...,Brandy
4,COURVOISIER VS COGNAC,Cognac,"Cognac, France",3.08,78,https://distiller.com/spirits/courvoisier-vs-c...,Brandy


In [26]:
print(df_alc_main_char.shape)
df_alc_main_char = df_alc_main_char.drop_duplicates().reset_index(drop = True)
print(df_alc_main_char.shape)

(740, 7)
(740, 7)


In [28]:
df_alc_main_char.to_csv(os.path.dirname(os.getcwd())+'/Datasets/URLs/all_brandies_main_20231210.csv', index = False)